In [4]:
# pip install langchain

In [1]:
import os
import openai
from string import Template
import json
from neo4j import GraphDatabase
import glob
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Neo4j configuration & constraints
from langchain.graphs import Neo4jGraph

neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
neo4j_user = os.getenv("NEO4J_USER")
neo4j_password = os.getenv("NEO4J_PASSWORD")
# graph = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))
 
graph = Neo4jGraph(
    url=neo4j_url,
    username=neo4j_user,
    password=neo4j_password
)
   

In [4]:
from langchain.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

In [6]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

In [12]:
import os
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate 


try :
  os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")
  llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

  def get_extraction_chain(
      allowed_nodes: Optional[List[str]] = None,
      allowed_rels: Optional[List[str]] = None
      ):
      prompt = ChatPromptTemplate.from_messages(
          [(
            "system",
            f"""# Knowledge Graph Instructions for GPT-4
  ## 1. Overview
  You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
  - **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
  - The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
  ## 2. Labeling Nodes
  - **Consistency**: Ensure you use basic or elementary types for node labels.
    - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
  - **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
  {'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
  {'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
  ## 3. Handling Numerical Data and Dates
  - Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
  - **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
  - **Property Format**: Properties must be in a key-value format.
  - **Quotation Marks**: Never use escaped single or double quotes within property values.
  - **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
  ## 4. Coreference Resolution
  - **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
  If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
  always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
  Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
  ## 5. Strict Compliance
  Adhere to the rules strictly. Non-compliance will result in termination.
            """),
              ("human", "Use the given format to extract information from the following input: {input}"),
              ("human", "Tip: Make sure to answer in the correct format"),
          ])
      return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

except Exception as exc:
  print(exc)
      

In [13]:

def extract_and_store_graph(
    document: str,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.run(document)
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    # Store information into a graph
    graph.add_graph_documents([graph_document])


# def extract_and_store_graph(document: str, nodes: Optional[List[str]] = None, rels: Optional[List[str]] = None) -> None:
#     extract_chain = get_extraction_chain(nodes, rels)
#     data = extract_chain.run(document)

#     graph_document = GraphDocument(
#         nodes=[map_to_base_node(node) for node in data.nodes],
#         relationships=[map_to_base_relationship(rel) for rel in data.rels],
#         source=document
#     )

#     # Store information into a graph (modify this part accordingly)
#     graph.add_graph_documents([graph_document])



In [20]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter

# Read the wikipedia article
raw_documents =  f"""

File Name: ALB-CPD-2021-EN
Year: 2021
Country Name: Albania
Country Code: ALB
Category: CPD
Document Title: UN Country programme document for Albania (2022–2026)
Exists?: Y
Publication Date: 30 August–2 September 2021
Start Year: 2022
End Year: 2026
Language: EN
------------------------------------------------

Executive Board of the
United Nations Development
Programme, the United Nations
Population Fund and the 
United Nations Office for
Project Services
Distr.: General
6 July 2021
Original: English
2109396X (E) 150721
Second regular session 2021
30 August–2 September 2021, New York
Item 5 of the provisional agenda
Country programmes and related matters
Country programme document for Albania (2022–2026)
#I. UNDP within the United Nations Sustainable Development Cooperation Framework
1. Albania, with its population of 2.9 million, is an upper-middle-income country with a 
high Human Development Index of 0.791,
Gender Development Index of 0.967 and Gender 
Inequality Index of 0.181. Women hold half of senior government positions. The economy 
is stable but underperforming, dependent on consumption and remittances, with low 
productivity and a limited tax base. Unemployment is structural and long-term at 11.8 per
cent with a gender pay gap of 6.6 per cent.
According to official figures, 38.3 per cent of 
the population suffer severe material deprivations and 23.4 per cent are at risk of falling into 
poverty (2018).
2. Albania faces multiple challenges that drive vulnerabilities, including weaknesses in 
rule of law, uneven institutional capacity, service gaps, limited investments in social services,
degradation of natural resources and few labour opportunities. 
3. The November 2019 earthquake and coronavirus disease (COVID-19) pandemic
highlighted the need to address disaster risk management and the socioeconomic impact of 
these events, particularly on vulnerable groups and micro, small and medium-sized
enterprises. The pandemic deepened gender inequalities, disproportionately affecting 
women, their livelihoods and access to essential health services, and increased incidents of 
gender-based violence.
4. In March 2020, the European Union opened accession negotiations. The country’s 
vision of sustainable development, the key drivers for reforms and political and public 
priorities, are accession to the European Union, sustainable economic growth and inclusive 
prosperity. The National Strategy for Development and Integration (NSDI) III is under 
preparation with interim nationalized Sustainable Development Goal indicators and targets.
5. This country programme reflects the comparative advantages of UNDP in supporting 
national reforms and upholding values of justice, human rights, equity and gender equality,
through coordinated multisectoral approaches that facilitate systemic reforms. Drawing on 
its reputation as a United Nations system “integrator” for the 2030 Agenda for Sustainable 
Development and trusted impartial partner, UNDP will continue to liaise with the 
Government, donors and international institutions to mobilize financing to support economic 
transformation and eradication of multidimensional poverty; and establish systems that 
address risks, disasters and social injustice within the Sustainable Development Goals. Work 
at the local level and partnerships within the United Nations and across society position 
UNDP well to support aspirations to join the European Union and attain the outcomes of the 
United Nations Sustainable Development Cooperation Framework (UNSDCF).
6. The summary7 of the previous country programme’s performance concluded that 
UNDP support to development remains highly relevant in Albania. It is viewed as an 
innovative thought leader drawing on international experiences, with the ability to convene 
different stakeholders underpinning its integrator role for the 2030 Agenda through: 
(a) technical expertise, offering integrated and cross-sectoral solutions, combining policy and 
regulatory advice with capacity development; (b) a strong and agile operational presence; 
(c) established multi-stakeholder partnerships on priorities for European Union accession 
and the Sustainable Development Goals; and (d) trusted relations with all institutions.
7 UNDP Albania country programme performance summary 2017-2021. The theory of change underpinning the country programme is that innovative 
approaches to inclusive, climate-friendly, gender-sensitive, sustainable growth in new 
competitive sectors, coupled with transparent, accountable governance systems, will boost 
jobs and businesses, identify investment opportunities, increase resilience and address 
interconnected vulnerabilities of climate, health and social injustice. UNDP will focus on 
local-level actions that together with national datasets and research, will inform national 
strategies and build forward better, particularly for those left behind.
8. Those left behind include women, children from low-income households, single parent 
households, persons with disabilities, the Roma and Egyptian communities, refugees, 
migrants, isolated rural communities, the elderly and bisexual, transgender, queer and 
intersex persons. Women who work in the care economy and in addition belong to minority 
groups and lack education, are the most vulnerable. Discrimination, stigmatization and 
gender-based violence are prevalent. Through a portfolio approach, UNDP will target these 
groups and increase social cohesion and social protection, in partnership with national and 
local governments, United Nations agencies and civil society. UNDP will partner with 
United Nations agencies on joint programmes related to leaving no one behind, strategic 
policy options for Sustainable Development Goal financing and ending violence against 
women.
9. The UNDP integration offer includes innovation, thematic expertise and financing for
renewables and energy efficiency in line with Government’s plans to provide safe, affordable 
and clean energy to citizens; and for raising the national ambitions to combat climate change 
and increase resilience to natural and human-induced risks. UNDP will promote nature-based 
solutions to climate change adaptation and mitigation, not only tackling the climate crisis, 
inequality, poverty and food insecurity, but also providing incentives for new jobs in the 
maritime, blue and eco-circular economies. Partnering with United Nations agencies, UNDP 
will implement programmes on a green circular economy and sustainable economic 
development. 
10. Economic growth is too low to converge with European Union living standards and 
achieve the Sustainable Development Goals. With United Nations agencies, UNDP will 
support employment measures for vulnerable groups, integrate small and medium-sized 
enterprises in global value chains, and support civil society to increase women’s access to 
careers in science, technology, engineering and mathematics. There are few incentives for 
adoption and innovation in digital technology, but a national digital data strategy and 
increasing digitalization are critical to accelerating the fourth industrial revolution. UNDP 
will focus on integrated solutions that address cross-cutting problems, i.e., identifying naturebased solutions that support the transition to a clean green economy, promote ecotourism and 
create new jobs in competitive sectors within the Western Balkan Regional Economic Area.
11. Key barriers to effective and efficient provision of services and that impact public trust 
are the lack of integrity and accountability of public institutions, slow progress on judicial 
reforms and corruption. Building on previous successes, UNDP will work with civil society 
and the government to improve human capacities and data systems for performance, 
transparency and planning. UNDP will work on the quality of services and rule of law, in 
close collaboration with United Nations partners.
12. With Albania positioning itself as a unifying actor for regional cooperation, security
and stability, and taking the initial steps to establish the national disaster risk management 
system as a requirement to join the European Union Civil Protection Mechanism, UNDP will 
continue to promote knowledge-sharing and relevant practices across the subregion. To the 
extent possible, UNDP will promote South-South and triangular cooperation in support of 
these efforts.
13. The report of the United Nations mainstreaming, acceleration and policy support 
mission to Albania and the United Nations country team reached consensus on three highly 
effective solutions for policy and programming to accelerate progress towards European 
Union accession and achieve the Sustainable Development Goals: (a) human capital 
development and social inclusion; (b) sustainable, resilient and green economic growth and 
resource management; (c) effective, people-centred governance, human rights, rule of law 
and gender equality. These are translated into four UNSDCF outcomes, fully adopted by 
UNDP and underpinned by five cross-cutting principles of human rights; gender equality;
inclusion and equality; transparent civic participation; and innovation.
14. The vision, value proposition and theory of change, which consolidates a number of 
past programmatic investments and proposes new transformative areas of action, were 
developed in consultation with over 200 partners representing civil society, private sector, 
academia, local and national governments, development partners and United Nations
agencies, and validated in a meeting with over 17 Deputy Ministers. 
#II. Programme priorities and partnerships 
15. This country programme reflects the country’s development priorities as articulated in 
the NSDI and the UNSDCF 2022-2026. In addition, it is aligned with the UNDP Strategic 
Plan and contributes to different European Union acquis chapters.
The Sustainable Development Goal targets and indicators and national datasets are integral to programme
outcomes and outputs. UNDP will contribute to these priorities, and application of its six 
signature solutions will help the country accelerate structural transformations. Specifically, 
UNDP will assist the country in the three main areas described below.
16. Human capital development and social inclusion. Albania needs well-functioning,
fully operational and funded social inclusion and protection policies. UNDP will work with 
national partners to promote social cohesion and strengthen social protection systems, 
services and financing mechanisms to increase coverage and quality for vulnerable groups.
With United Nations agencies, UNDP will ensure civic engagement and capacities for 
dialogue and implementation of social care plans. 
17. COVID-19 generated a sense of urgency to operationalize national policies, strengthen 
local government capacities and design and fund inclusive, gender-responsive and evidencebased services. To reach those most left behind, UNDP will expand services at municipal 
and national levels in education, health, social care, employment, vocational training and 
housing. UNDP, with the European Union, will continue investments in the education sector
affected by the 2019 earthquake, adopting European Union standards and expanding 
information and communication technology measures as a post-COVID-19 recovery 
strategy. UNDP will technically lead implementation of the United Nations COVID-19 
socioeconomic recovery plan.
18. Sustainable, resilient and green economic growth and resource management.
UNDP will support policies that accelerate an inclusive, productive and sustainable 
economy, enhance climate resilience and adaptation planning, and transition into a clean 
green economy, in line with the Government’s national strategies. This will be done through:
(a) strengthening governance mechanisms to address climate change, biodiversity loss, 
pollution and land degradation; (b) promoting ecotourism and circular bioeconomy;
(c) promote a “source-to-sea” approach, and strengthen transboundary cooperation for water 
resources management in the Drini River Basin; (d) address the skills mismatch of youth and 
women for innovation and digitalization; and (e) support value chains of micro, small and 
medium-sized enterprises, decent jobs and inclusive employment, particularly for persons 
with disabilities and women. UNDP will support industries to become more energy-efficient,
increase the potential of the forestry sector for removal of greenhouse gasses, and support 
the country’s green growth trajectory. UNDP will support promotion of nature-based tourism 
that sustains livelihoods and protected areas, while improving ecosystem services and 
financial sustainability.
19. UNDP will support entities to have improved systems and capacities to design and 
implement innovative policies, particularly in relation to qualitative skills development that 
promotes a digital economy, entrepreneurship and competitive green businesses. UNDP will 
focus on integrated multisectoral solutions that provide: the necessary skills, vocational 
education and training; access to decent jobs and innovative technology; sustainable 
livelihoods; and business opportunities. With young people (aged 15-24) comprising 25 per 
cent of the population, UNDP will continue to promote youth as productive human capital 
and partner with labour-market stakeholders to boost employment and entrepreneurship. 
UNDP will consolidate reforms, strengthen capacities, address skills mismatch and increase 
labour force participation especially for women and young people, persons with disabilities, 
Roma and Egyptian communities and the long-term unemployed. 
20. With inclusive growth being a key driver for reducing vulnerability and building 
resilience, particularly for the poor and most disadvantaged, UNDP will continue to support 
policies and measures leading to an expedited economic recovery and increased 
competitiveness and productivity of (female-led) micro, small and medium-sized enterprises. 
UNDP will work towards improved environments for digital innovation through support for 
data-driven innovations, bringing higher education closer to the labour market and 
supporting new course curricula for future skills. Increasing vocational education and 
training to match labour market demands and integrating small and medium-sized enterprises 
in global value chains are additional strategies. A vision and road maps for green growth and 
a blue economy will be formulated, thus ensuring growth that protects natural resources.
21. To contribute to a strengthened risk and disaster management system, UNDP will assist 
the Government in driving policies and actions that focus on prevention and reduction of 
disaster-related risks, increasing resilience of infrastructures, ecosystems and society, and 
enabling adaptive capacities, all as a means to reduce the long-term vulnerability of the 
population and protect development investments, particularly in at-risk areas. With a 
“knowing better, losing less” approach, UNDP will focus on local-level risk assessments, 
utilizing data and knowledge in parallel with prevention measures. Raising the national 
ambition to combat climate change, increasing resilience to natural and human-induced risk, 
and gender-responsive disaster strategies remain priorities.
22. Effective, people-centred governance, human rights, rule of law and gender 
equality. UNDP will broaden partnerships for a stronger good governance framework, in 
line with the National Public Administration Reform Strategy. UNDP will foster a citizencentred, gender-sensitive, inclusive approach, while further developing institutional
capacities in policymaking and implementation, promoting a culture of participation and 
dialogue for development, and increasing transparency, integrity and accountability. Based 
on its initial successful experiences in rolling out integrity planning in municipalities, UNDP 
will build national capacities to bring to scale and monitor this initiative.
23. Innovation and digitalization will be explored to establish robust, modernized and 
efficient service delivery. UNDP will support integrated citizen-centred services and further 
expand municipal one-stop-shop networks to reach remote, mostly rural areas. 
24. UNDP will support legal, policy and institutional reforms related to human rights 
commitments and civic engagement; capacity development for policymaking and 
monitoring; and access to justice for vulnerable people. UNDP will continue to support the 
People’s Advocate, and the Commissioners for Information and Data Protection and 
Protection from Discrimination. UNDP will continue to work towards ending all forms of 
domestic, sexual and gender-based violence and unfavourable social norms and gender 
stereotypes, supporting coordinated referral mechanisms and integrated services, safe 
accommodation and accessible emergency and specialized support. It will continue to support the Government to address structural and systemic barriers to gender equality and 
women’s empowerment in policy formulation and implementation, and to generate quality 
gender-disaggregated data for targeted, informed investments and quantifying of results.
25. The programme will support data availability and a more data-driven culture for
strengthened sectoral analysis, evidence-based policymaking and promotion of a culture of 
accountability and performance assessment. Policy analysis and formulation will include a 
gender equality focus and gender-disaggregated data. UNDP will engage gender champions 
and support increased civic engagement and dialogue platforms.
26. UNDP will continue to work closely with existing donors, vertical funds and United 
Nations agencies, diversifying funding sources by partnering with new donors including on
an integrated national finance framework, while broadening engagement with the private 
sector, academia, civil society, national human rights institutions and the media.
#III. Programme and risk management
27. This country programme document (CPD) outlines UNDP’s contributions to national 
results and serves as the primary unit of accountability to the Executive Board for results 
alignment and resources assigned to the programme at country level. Accountabilities of 
managers at the country, regional and headquarter levels with respect to country programmes 
are prescribed in the organization’s programme and operations policies and procedures and 
internal control framework.
28. The programme will be nationally executed. If necessary, national execution may be 
replaced by direct execution for part or all of the programme to enable response to force 
majeure. The harmonized approach to cash transfers will be used in a coordinated fashion 
with other United Nations agencies to manage financial risks. Cost definitions and 
classifications for programme and development effectiveness will be charged to the 
concerned projects.
29. UNDP will participate in joint Government–United Nations coordination mechanisms. 
Programme and project boards will be established in line with the social and environmental 
standards and accountability mechanism. 
30. The main risk to the programme relates to the ongoing impact of COVID-19 and an 
unforeseen impact on the capacity to implement the programme or achieve its results.
Possible changes in the process and requirements for European Union accession may impact 
national development policies. Natural or man-made hazards, with insufficient institutional 
and local preparedness to respond, can impact the ability of UNDP to support Albania in 
delivering its development priorities.
31. Risks will be managed through coordination and partnerships; advocacy and policy 
dialogue; regular monitoring of risks, project and results; and updating of contingency plans 
for the country office and national emergencies.
#IV. Monitoring and evaluation
32. The monitoring and evaluation plan will be implemented following endorsement of the 
CPD by the Executive Board. The results framework is aligned with indicators related to 
European Union accession, the NSDI and the national indicator framework for the 
Sustainable Development Goals. This demonstrates a clear linkage between the support 
provided by UNDP, the programme outcomes and sustainable development results.
33. The planned evaluations will contribute to accountability, learning and achievement of 
strategic results. They meet all the required criteria, provide sufficient and balanced coverage 
of the programme’s areas of engagement and emphasize national ownership and capacity 
development. Planned project evaluations relate to ongoing projects. Results of new projects
will be captured in the midterm review or in future project evaluations added during yearly reviews of the evaluation plan. Socioeconomic, political and environmental risks were 
considered during the formulation of the evaluation plan. Ethical practices and lessons 
learned shape policy advice, advocacy and project design and implementation. An 
independent country programme evaluation will be conducted in 2025.
34. UNDP will employ national monitoring systems, official statistical data and reports 
from line ministries, development partners and research institutions to measure its 
contribution to national results. Other data, perception surveys and reports will be used to 
assess results and behavioural change. The National Statistical Office and other partners will 
receive support to continue improving data quality. Data for baselines and targets will be 
disaggregated by sex, geography and income and increasingly identify those left behind. The 
programme will reinforce innovative tools to manage data collection and analysis at the 
country programme outcome and output levels.

"""

# WikipediaLoader(query="Walt Disney").load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=24)
# Only take the first the raw_documents
documents = text_splitter.create_documents([raw_documents])
# documents
 

In [ ]:
from tqdm import tqdm
for i, d in tqdm(enumerate(documents), total=len(documents)):
    extract_and_store_graph(d)
print(documents)

In [25]:
# from concurrent.futures import ThreadPoolExecutor
# from tqdm import tqdm

# # Define a function that performs the API call and processing for a single document
# def process_document(d):
#     extract_and_store_graph(d)  # Replace this with your function to process a single document

# # Your documents list

# # Define the number of workers/threads (adjust as needed)
# num_workers = 1000  # Number of threads/workers

# # Process documents in parallel using ThreadPoolExecutor
# with ThreadPoolExecutor(max_workers=num_workers) as executor:
#     list(tqdm(executor.map(process_document, documents), total=len(documents)))


 

  0%|          | 0/5 [25:15<?, ?it/s]


In [ ]:

# Delete the graph
# graph.query("MATCH (n) DETACH DELETE n")